In [ ]:
import xarray as xr
import hvplot.xarray
import netCDF4
import pandas as pd
import netCDF4
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from metpy.plots import SkewT
from metpy.units import pandas_dataframe_to_unit_arrays, units
from datetime import datetime, timedelta
from IPython.display import display
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure, show
from bokeh.models import Title, CustomJS, Select, TextInput, Button, LinearAxis, Range1d, FuncTickFormatter
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.palettes import Category10
import warnings
import itertools 
import holoviews as hv 
from holoviews import dim, opts
import hvplot.pandas
hv.extension('bokeh', 'matplotlib')
warnings.filterwarnings('ignore')
output_notebook()
%matplotlib inline

In [ ]:
import os
try:
    os.system('source ~/.bash_profile')
    CL = os.environ.get("QA_CL")
except:
    pass

In [ ]:
if CL == 'command_line_mode':
    try:
        project = os.environ.get('QA_PROJ')
        flight = os.environ.get('QA_FLIGHT')
    except:
        pass
else:
    try:
        #######################################################################
        ####### change project and flight below if running interactively ######
        #######################################################################
        #project = 'wecan'
        #flight = 'rf18'
        project = 'CGWAVES'
        flight = 'tf02'
        #######################################################################
    except:
        pass
print('Project: ' + project)
print('Flight: ' + flight)

In [ ]:
#Set the DATA_DIR os environment variable to the current working directory if it is not already set
if 'DATA_DIR' not in os.environ:
    os.environ['DATA_DIR'] = os.getcwd()

In [ ]:
# use the user provided project and flight information to build the path and store the data file as an object
########################################################################
# update filepath based on where the netcdf files are located
########################################################################
yr = '2024'
filepath = os.environ.get('DATA_DIR')+'/'+project +'/'
input_file = project + flight + '.nc'
nc = netCDF4.Dataset(filepath + input_file, mode='r')

# try to get global attributes from the netcdf file if they are present
# determine preliminary or final status
try:
    proc_status = nc.getncattr('WARNING')
    print(proc_status)
except:
    proc_status = 'final'

# determine the NIDAS version
try:
    nidas = nc.getncattr('NIDASrevision')
    print('NIDAS version: ' + nidas)
except Exception as e:
    print(e)

# determine the NIMBUS version
try:
    nimbus = nc.getncattr('RepositoryRevision')
    print('NIMBUS version: ' + nimbus)
except Exception as e:
    print(e)

# determine the processing date and time
try:
    proc_date = nc.getncattr('date_created')
    print('Processing Date & Time: ' + proc_date)
except Exception as e:
    print(e)

In [ ]:
# sometimes the netcdf4 api produces an issue with big-endian buffer on little-endian compiler
byte_swap = False

# create empty placeholders for asc, histo_asc and units
asc = {}
histo_asc = {}
units = {}
cellsize = {}

# use the netcdf4 api to get the netcdf data into a dataframe
try:
    
    # loop over keys in netCDF file and organize
    for i in nc.variables.keys():
        dims = str(nc.variables[i].dimensions)
        
        # this if section retrieves data that has time dimension only
        # this section retrieves data that has a size distribution dimension in addition to time
        if "sps1" in dims:
            histo_output = nc.variables[i][:, 0, :]
            # sometimes the netcdf4 api produces an issue with big-endian buffer on little-endian compiler
            if byte_swap == True:
                histo_output = histo_output.byteswap().newbyteorder()
            else:
                pass
            histo_asc[i] = pd.DataFrame(histo_output)
            
            # this try / except block accommodates size distribution data that has an attribute CellSizes
            try:
                cellsize = nc.variables[i].getncattr('CellSizes')
                histo_asc[i].columns = pd.MultiIndex.from_tuples(zip(histo_asc[i].columns, cellsize))
            except Exception as e:
                histo_asc[i].columns = pd.MultiIndex.from_tuples(zip(histo_asc[i].columns, histo_asc[i].columns))
        else:
            pass    

    
    # concatenate the histogram data
    histo_asc = pd.concat(histo_asc, axis=1, ignore_index=False)
    histo_asc.columns = histo_asc.columns.droplevel(1)
    colors = itertools.cycle(Category10[6])
    
    
except Exception as e:
    print(e)

In [ ]:
nc.close()

In [ ]:
df = xr.open_dataset(filepath+input_file) #open the dataset as an xarray
ds =df.where(df.GGSPD >60, drop = True) #filter the data to not include time on the ground
x_width = 1200
y_height = 400

In [ ]:
# create lists of the histogram variables
try:
    histovar_list = list(histo_asc.columns.levels[0])
    histovar_list = [var for var in histovar_list if not var.endswith('VXL')]
    histobin_list = []
except:
    pass

# loop over the length of the histogram variable list
try:
    for i in range(len(histovar_list)):
        histobin_list.append(max(list(histo_asc[histovar_list[i]]))+1)
except:
    pass

# create list of histogram units
try:
    histo_units = []
    for i in histovar_list:
        histo_units.append(nc.variables[i].getncattr('units'))
except:
    pass

In [ ]:
def plot_histogram(ds, var, i):
    try:
        # Check if the variable exists in the dataset
        if var not in ds:
            print(f"{var} not in dataset")
            return
            
        # Get dimensions and check if we have the right ones
        dims = ds[var].squeeze().dims

        # Find the bin dimension (non-time dimension)
        try:
            vname = [item for item in dims if 'Time' not in item][0]
        except IndexError:
            print(f"{var} doesn't have a non-time dimension for binning")
            return
            
        # Check if we have any valid data
        if ds[var].isnull().all():
            print(f"{var} contains only NaN values - skipping plot")
            return
            
        # Get plot limits with safety checks
        max_z = float(ds[var].to_numpy().max())
        if not np.isfinite(max_z) or max_z <= 0:
            # Try to find any non-NaN values to plot
            non_nan_data = ds[var].where(~np.isnan(ds[var]))
            if non_nan_data.count() > 0:
                max_z = float(non_nan_data.max().values)
                print(f"Adjusted {var} to use non-NaN maximum: {max_z}")
            else:
                print(f"{var} has invalid maximum value: {max_z} - skipping plot")
                return
            
        min_y = 1 if ds[var][vname][0] == 0 else float(ds[var][vname][0])
        if min_y <= 0:
            min_y = 1  # Ensure positive for log scale
            
        # Set up plot labels
        ylab = "Bin [#]" if min_y == 1 else "Cell Size [um]"
        
        # Create plot with safe options - replace NaN with 0
        plot_ds = ds[var].squeeze().fillna(0).assign_coords({vname: ds[vname]})
        
        heatmap = plot_ds.hvplot.quadmesh(
            cmap='gist_ncar', x='Time', width=x_width, height=y_height,
            ylabel=ylab, xlabel='Time [UTC]', 
            title=f"{var} Units: {histo_units[i]}",
            y=vname, clim=(0, max_z), ylim=(min_y, None), logy=True,
        )
        
        # Render and show the plot
        bokeh_plot = hv.render(heatmap)
        show(bokeh_plot)
        
    except IndexError:
        print(f"{var} does not have vector dimension")
    except KeyError:
        print(f"{var} not in file")
    except Exception as e:
        print(f"Error plotting {var}: {e}")

In [ ]:
for i, var in enumerate(histovar_list):
    plot_histogram(ds,var,i)